In [ ]:
import os 
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import sys
sys.path.append("..")

from make_dir import mkdir
from load_yaml import get_yaml

import models.micro_macro_net as solutions
import equations.micro_macro_eqn as equation

from micro_macro_dataset import Sampler
import micro_macro_solver as solver 

import matplotlib.pyplot as plt
import time


In [ ]:
# load config
current_path = os.path.abspath(".")
yaml_path = os.path.join(current_path, "micro_macro.yaml")
Config = get_yaml(yaml_path)

# load reference data
absolute_path = os.path.abspath("..")
ref_path = os.path.join(absolute_path, "data/ref_kn1e-8.npz")
ref_rho = torch.Tensor(np.load(ref_path)["macro_frames"]).to('cpu').reshape((-1, 1)) # shape: (100, 1)


In [ ]:
time_dimension = Config["physical_config"]["time_dimension"]
space_dimension = Config["physical_config"]["space_dimension"]
velocity_dimension = Config["physical_config"]["velocity_dimension"]
rho_d_in = time_dimension + space_dimension
layers_rho = Config["model_config"]["units_rho"]
g_d_in = time_dimension + space_dimension + velocity_dimension
layers_g = Config["model_config"]["units_g"]

# build neural networks for rho, g
Model_rho = "solutions.Model_rho_" + \
    "{}".format(Config["model_config"]["neural_network_type"])
Model_rho = eval(Model_rho)

model_rho = Model_rho(input_size = rho_d_in, layers = layers_rho, output_size = 1)

Model_g = "solutions.Model_g_" + \
    "{}".format(Config["model_config"]["neural_network_type"])
Model_g = eval(Model_g)

model_g = Model_g(input_size = g_d_in, layers = layers_g, output_size = 1)

device_ids = Config["model_config"]["device_ids"]
device = torch.device("cuda:{:d}".format(device_ids[0]) if torch.cuda.is_available() else "cpu")

if torch.cuda.device_count() > 1:
    model_rho = nn.DataParallel(model_rho, device_ids = device_ids)    
    model_g = nn.DataParallel(model_g, device_ids = device_ids)
    
model_rho.to(device)
model_g.to(device)


In [ ]:
# number of paramerters
rho_param_num = sum(neural.numel() for neural in model_rho.parameters())
g_param_num = sum(neural.numel() for neural in model_g.parameters())
print("Number of paramerters for networks u is: {:6d} and {:6d}. ".format(rho_param_num, g_param_num))


In [ ]:
solutions.Xavier_initi(model_rho)
solutions.Xavier_initi(model_g)


In [ ]:
# Set optimizer and learning rate decay
optimizer = optim.Adam([
    {'params': model_rho.parameters()},
    {'params': model_g.parameters()},
],  lr=Config["model_config"]["lr"])

scheduler = lr_scheduler.StepLR(
    optimizer, Config["model_config"]["stage_num"], Config["model_config"]["decay_rate"])


In [ ]:
Sol = model_rho, model_g
eqn = equation.MicroMacro(config = Config, sol = Sol)

Iter = Config["model_config"]["iterations"] 
regularizers = Config["model_config"]["regularizers"]

loss_record, error_record = np.array([[]]).T, np.array([[]]*1).T

mkdir(file_dir = "./model_saved")
mkdir(file_dir = "./record")
mkdir(file_dir = "./figure")

time_start = time.time()
print('Begin training.')
print('')
for it in range(Iter):
    
    sampler = Sampler(Config)
    trainloader = [sampler.interior(), sampler.boundary(), sampler.initial()]
        
    risk, error = solver.train_step(sol = Sol,
                                    trainloader = trainloader, 
                                    equation = eqn,  
                                    regularizers = regularizers,
                                    optimizer = optimizer, 
                                    scheduler = scheduler,
                                    ref = ref_rho)
    
    loss = risk["total_loss"]
    res_micro_eqn = risk["micro"]
    res_macro_eqn = risk["macro"]
    res_bc_f = risk["bc_f"]
    res_ic_f = risk["ic_f"]
    error = error["error"] 

    error = np.array(error, dtype=float).reshape(1, -1)
    loss_record = np.concatenate((loss_record, loss*np.ones((1, 1))), axis=0)
    error_record = np.concatenate((error_record, error), axis=0)

    lr = optimizer.state_dict()['param_groups'][0]['lr']
    
    if it % 100 == 0:
    
        print("[Iter: {:6d}/{:6d} - lr: {:.2e} and Loss: {:.2e}]".format(it + 1, Iter, lr, loss))
        print("[Error for density: {:.2e}]".format(float(error[:, 0])))
        print("[Eqn micro: {:.2e}, macro: {:.2e}, Boundary: {:.2e}, Initial: {:.2e}]".format(res_micro_eqn, res_macro_eqn, res_bc_f, res_ic_f))

        
    if np.max(error) < 1e-3:
        print("Iteration step: ", it)
        break

np.savez("./record/result.npz",
         loss=loss_record,
         error=error_record[:, 0])

solutions.save_param(model_rho, path = './model_saved/model_rho_params.pkl')
solutions.save_param(model_g, path = './model_saved/model_g_params.pkl')

print("")
print("Finished training.")
time_end = time.time()
print("Total time is: {:.2e}".format(time_end - time_start), "seconds")


In [ ]:
# # load model
solutions.load_param(model_rho, './model_saved/model_rho_params.pkl')
solutions.load_param(model_g, './model_saved/model_g_params.pkl')

Sol = [model_rho, model_g]

In [ ]:
data_size = 100
ref_x = torch.Tensor(np.linspace(0.005, 0.995, data_size).reshape((data_size, 1))).to(device)
ref_t = 0.1 * torch.ones((ref_x.shape[0], 1)).to(device)
approx_rho = eqn.rho(sol=Sol, inputs=[ref_t, ref_x]).cpu().detach().numpy()

plt.style.use("seaborn-dark") 
fig = plt.figure()
plt.plot(ref_x.cpu().detach().numpy() , approx_rho, color = 'r', marker = 'x', linewidth = 0.0, markersize = 5, markevery = 2, label = 'APNN(t = 0.1)')
plt.plot(ref_x.cpu().detach().numpy() , ref_rho, color = 'k', linewidth = 1.0, markersize = 10, label = 'Ref(t = 0.1)')
plt.grid()
plt.legend()
plt.xlabel(r"x")
plt.ylabel(r"$\rho$")
plt.title(r"$\rho,$ ref at $t = 1$")
# plt.savefig('./figure/example_micro_macro.pdf')
plt.show()

In [ ]:
# # jupyter notebook to python
# try:   
#     !jupyter nbconvert --to python example.ipynb
# except:
#     pass